In [17]:
import torch 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

%cd ../

/workspace/Walk_Video_PyTorch


In [18]:
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()

In [19]:
from pytorch_lightning import seed_everything

seed_everything(42, workers=True)

Global seed set to 42


42

In [20]:
from main import get_parameters

opt, unknown = get_parameters()
opt.num_workers = 8
opt.batch_size = 4
opt.gpu_num = 1

opt.version = '0821_2_16'
opt.model = "resnet"
opt.model_depth = 50
opt.model_class_num = 1

opt.clip_duraion = 1
opt.uniform_temporal_subsample_num = 16
opt.version = opt.version + '_' + opt.model + '_depth' + str(opt.model_depth)


In [21]:
from utils.utils import get_ckpt_path

model = WalkVideoClassificationLightningModule(opt)

# get last ckpt path
# ckpt_path = get_ckpt_path(opt)

ckpt_path = '/workspace/Walk_Video_PyTorch/logs/resnet/0902_1_16_resnet_depth50/checkpoints/epoch=7-val_loss=0.89-val_acc=0.7215.ckpt'

# model = WalkVideoClassificationLightningModule.load_from_checkpoint(ckpt_path)
# model.load_state_dict(weight)
model = model.load_from_checkpoint(ckpt_path)

model.eval()

# clear_output()
print(ckpt_path)


Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


/workspace/Walk_Video_PyTorch/logs/resnet/0902_1_16_resnet_depth50/checkpoints/epoch=7-val_loss=0.89-val_acc=0.7215.ckpt


In [22]:
from dataloader.data_loader import WalkDataModule
from pytorch_lightning import loggers as pl_loggers 

# load test dataset 
module = WalkDataModule(opt)
module.setup()
test_data = module.test_dataloader()

# for the tensorboard
tb_logger = pl_loggers.TensorBoardLogger(save_dir="/workspace/Walk_Video_PyTorch/project/tests/logs", name=opt.model, version=opt.version)


In [32]:

trainer = Trainer(
    accelerator="auto",
    devices=1,
    gpus=opt.gpu_num,
    logger=tb_logger,
    max_epochs=1,
    # deterministic=True,
)

trainer.test(dataloaders=module, model=model, ckpt_path=ckpt_path)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at /workspace/Walk_Video_PyTorch/logs/resnet/0902_1_16_resnet_depth50/checkpoints/epoch=7-val_loss=0.89-val_acc=0.7215.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]
Loaded model weights from checkpoint at /workspace/Walk_Video_PyTorch/logs/resnet/0902_1_16_resnet_depth50/checkpoints/epoch=7-val_loss=0.89-val_acc=0.7215.ckpt


Testing: 0it [00:00, ?it/s]

tensor(0.7426, device='cuda:0')


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7397260069847107     │
│  test_average_precision   │            nan            │
│         test_loss         │    0.7818960547447205     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.7818960547447205,
  'test_acc': 0.7397260069847107,
  'test_average_precision': nan}]

In [33]:
input_data = next(iter(test_data))


In [34]:
input_data['video'].shape

input_data["video_name"]

['20211221_ASD_lat_V1-0007.mp4',
 '20210518_1_ASD_lat_V1-0006.mp4',
 '20210812_DHS_lat_V1-0023.mp4',
 '20210309_ASD_lat__V1-0037.mp4']

In [35]:
preds = model(input_data['video'])

In [36]:
real_label = input_data['label']
pred_label = preds

real_label, pred_label


(tensor([0, 0, 1, 0]),
 tensor([[-4.1151],
         [-8.6586],
         [11.3148],
         [-2.1958]], grad_fn=<ViewBackward0>))

In [37]:
from torch.nn.functional import sigmoid, tanh, softmax

sigmoid(pred_label), tanh(pred_label), softmax(pred_label)

/tmp/ipykernel_1596399/3629341915.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sigmoid(pred_label), tanh(pred_label), softmax(pred_label)


(tensor([[1.6062e-02],
         [1.7360e-04],
         [9.9999e-01],
         [1.0013e-01]], grad_fn=<SigmoidBackward0>),
 tensor([[-0.9995],
         [-1.0000],
         [ 1.0000],
         [-0.9755]], grad_fn=<TanhBackward0>),
 tensor([[1.],
         [1.],
         [1.],
         [1.]], grad_fn=<SoftmaxBackward0>))

In [45]:
sigmoid_pred = sigmoid(pred_label)

sigmoid_pred.topk(k=1)

/usr/local/lib/python3.9/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.return_types.topk(
values=tensor([[1.6062e-02],
        [1.7360e-04],
        [9.9999e-01],
        [1.0013e-01]], grad_fn=<TopkBackward0>),
indices=tensor([[0],
        [0],
        [0],
        [0]]))

In [47]:
from utils.metrics import get_Accuracy, get_Average_precision

accuracy = get_Accuracy(1)
AP = get_Average_precision(1)

accuracy(preds.squeeze(), input_data['label'])
AP(preds.squeeze(), input_data['label'])

tensor(0.6389)